# Reproduction Notebook

This notebook demonstrates how to load the experiment results and visualize the null distributions against the observed metrics. It serves as a proof of reproducibility for the Voynich Modular-23 experiment.

In [ ]:
import json
import pathlib
import matplotlib.pyplot as plt
import numpy as np

# Find the latest result file
results_dir = pathlib.Path("../results")
result_files = sorted(results_dir.glob("run_*.json"), reverse=True)

if not result_files:
    raise FileNotFoundError("No result files found in results/ directory. Run run_experiment.py first.")

latest_result = result_files[0]
print(f"Loading results from: {latest_result}")

with open(latest_result) as f:
    data = json.load(f)

metrics = data["metrics"]

## 1. Structure Analysis (Gzip Compression)

We compare the Gzip compressed size of the decoded Voynich text against 10,000 random character shuffles. A significantly smaller size indicates non-random structural redundancy.

In [ ]:
if "null_samples" in metrics["gzip"]:
    obs_gzip = metrics["gzip"]["observed"]
    null_gzip = metrics["gzip"]["null_samples"]

    plt.figure(figsize=(10, 6))
    plt.hist(null_gzip, bins=50, color='gray', alpha=0.7, label='Null (Shuffled Text)')
    plt.axvline(obs_gzip, color='red', linestyle='--', linewidth=2, label=f'Observed ({obs_gzip})')
    plt.title(f"Gzip Compression Size\np-value: {metrics['gzip']['p_value_smaller']:.5f}")
    plt.xlabel("Compressed Size (bytes)")
    plt.ylabel("Frequency")
    plt.legend()
    plt.show()
else:
    print("Raw null samples for Gzip not found (experiment run with --no-raw?)")

## 2. Linguistic Affinity (Trigram Similarity)

We compare the Trigram Cosine Similarity to Latin against 10,000 random alphabet permutations. A significantly higher similarity suggests the mapping produces Latin-like clusters.

In [ ]:
if "null_samples" in metrics["trigram_cosine"]:
    obs_sim = metrics["trigram_cosine"]["observed"]
    null_sim = metrics["trigram_cosine"]["null_samples"]

    plt.figure(figsize=(10, 6))
    plt.hist(null_sim, bins=50, color='gray', alpha=0.7, label='Null (Random Alphabet)')
    plt.axvline(obs_sim, color='blue', linestyle='--', linewidth=2, label=f'Observed ({obs_sim:.4f})')
    plt.title(f"Trigram Cosine Similarity to Latin\np-value: {metrics['trigram_cosine']['p_value_greater']:.5f}")
    plt.xlabel("Cosine Similarity")
    plt.ylabel("Frequency")
    plt.legend()
    plt.show()
else:
    print("Raw null samples for Trigrams not found.")

## Interpretation

If the observed lines (colored dashed) fall significantly outside the gray distributions, the result is statistically significant. 

- **Left of the distribution (Gzip):** The text is more structured than random noise.
- **Right of the distribution (Trigrams):** The text is more similar to Latin than a random substitution cipher would be.